# Deploy Trained Smolvla Policy

<img src="./media/rollout3.gif" width="480" height="360">

Deploy trained policy in simulation.

In [1]:
!pip install transformers==4.50.3
!pip install num2words
!pip install accelerate
!pip install safetensors>=0.4.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


### [Optional] Download Dataset

In [2]:
'''
If you want to use the collected dataset, please download it from Hugging Face.
'''
# !git clone https://huggingface.co/datasets/Jeongeun/omy_pnp_language

'\nIf you want to use the collected dataset, please download it from Hugging Face.\n'

## Step 2. Train Model

In [3]:
# !python train_model.py --config_path smolvla_omy.yaml

## Step 3. Deploy

In [4]:
# Dataset imports - moved from lerobot.common.datasets
from lerobot.datasets.lerobot_dataset import LeRobotDataset, LeRobotDatasetMetadata
from lerobot.datasets.utils import write_json, serialize_dict, dataset_to_policy_features
from lerobot.datasets.factory import resolve_delta_timestamps

# SmolVLA policy imports - moved from lerobot.common.policies
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy

# Config types - moved from lerobot.configs
from lerobot.configs.types import FeatureType

# Standard imports remain the same
import numpy as np
import torch
from PIL import Image
import torchvision


/home/dragon/anaconda3/envs/mujoco_vla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = 'cuda'

In [6]:
try:
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./demo_data_language')
except:
    dataset_metadata = LeRobotDatasetMetadata("omy_pnp_language", root='./omy_pnp_language')
features = dataset_to_policy_features(dataset_metadata.features)
output_features = {key: ft for key, ft in features.items() if ft.type is FeatureType.ACTION}
input_features = {key: ft for key, ft in features.items() if key not in output_features}
# Policies are initialized with a configuration class, in this case `DiffusionConfig`. For this example,
# we'll just use the defaults and so no arguments other than input/output features need to be passed.
# Temporal ensemble to make smoother trajectory predictions
cfg = SmolVLAConfig(input_features=input_features, output_features=output_features, chunk_size= 5, n_action_steps=5)
delta_timestamps = resolve_delta_timestamps(cfg, dataset_metadata)

In [7]:
# 加载本地训练的policy（专门用于移除红块任务）
local_policy = SmolVLAPolicy.from_pretrained("DragonHu/smolvla_UR52_empty_plat_block_fixed_v2", dataset_stats=dataset_metadata.stats)
# local_policy = SmolVLAPolicy.from_pretrained('./ckpt/smolvla_omy/checkpoints/last/pretrained_model', dataset_stats=dataset_metadata.stats)
local_policy.to(device)
local_policy.eval()  # 设置为评估模式
print("✅ 本地policy加载完成 - 专门用于移除红块任务")

# 加载云端policy（专门用于放置蓝杯任务）
try:
    cloud_policy = SmolVLAPolicy.from_pretrained("Jeongeun/omy_pnp_smolvla", config=cfg, dataset_stats=dataset_metadata.stats)
    cloud_policy.to(device)
    cloud_policy.eval()  # 设置为评估模式
    print("✅ 云端policy加载完成 - 专门用于放置蓝杯任务")
except Exception as e:
    print(f"❌ 加载云端policy失败: {e}")
    cloud_policy = None

print("🎯 Policy配置完成：")
print("   - 第一阶段（移除红块）：使用 local_policy")
print("   - 第二阶段（放置蓝杯）：使用 cloud_policy")

Reducing the number of VLM layers to 16 ...
✅ 本地policy加载完成 - 专门用于移除红块任务
Reducing the number of VLM layers to 16 ...
✅ 云端policy加载完成 - 专门用于放置蓝杯任务
🎯 Policy配置完成：
   - 第一阶段（移除红块）：使用 local_policy
   - 第二阶段（放置蓝杯）：使用 cloud_policy


In [21]:
from mujoco_env.y_env2_removeBlock_Pnp_mug import SimpleEnvRemoveBlock_pnp_mug
xml_path = './asset/scene_remove_block.xml'
PnPEnv = SimpleEnvRemoveBlock_pnp_mug(xml_path, action_type='joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[22]
 [0/22] [world] mass:[0.00]kg
 [1/22] [front_object_table] mass:[1.00]kg
 [2/22] [camera] mass:[0.00]kg
 [3/22] [camera2] mass:[0.00]kg
 [4/22] [camera3] mass:[0.00]kg
 [5/22] [link1] mass:[2.06]kg
 [6/22] [link2] mass:[3.68]kg
 [7/22] [link3] mass:[2.39]kg
 [8/22] [link4] mass:[1.40]kg
 [9/22] [link5] mass:[1.40]kg
 [10/22] [link6] mass:[0.65]kg
 [11/22] [camera_center] mass:[0.00]kg
 [12/22] [tcp_link] mass:[0.32]kg
 [13/22] [rh_p12_rn_r1] mass:[0.07]kg
 [14/22] [rh_p12_rn_r2] mass:[0.02]kg
 [15/22] [rh_p12_rn_l1] mass:[0.07]kg
 [16/22] [rh_p12_rn_l2] mass:[0.02]kg
 [17/22] [body_obj_plate_11] mass:[0.00]kg
 [18/22] [object_plate_11] mass:[0.10]kg
 [19/22] [body_obj_mug_6] mass:[0.00]kg
 [20/22] [object_mug_6] mass:[0.08]kg
 [21/22] [body_obj_block_red] mass:[0.30]kg
body_total_mass:

In [22]:
from torchvision import transforms
# Approach 1: Using torchvision.transforms
def get_default_transform(image_size: int = 224):
    """
    Returns a torchvision transform that:
     Converts to a FloatTensor and scales pixel values [0,255] -> [0.0,1.0]
    """
    return transforms.Compose([
        transforms.ToTensor(),  # PIL [0–255] -> FloatTensor [0.0–1.0], shape C×H×W
    ])

In [23]:
step = 0
PnPEnv.reset(seed=0)
local_policy.reset()
local_policy.eval()
if cloud_policy is not None:
    cloud_policy.reset()
    cloud_policy.eval()

save_image = True
IMG_TRANSFORM = get_default_transform()

print("🚀 开始执行双任务：1. 移除红块 2. 放置蓝杯")
print("📋 Policy配置确认：")
print(f"   - 第一阶段（移除红块）：{'✅ 本地policy' if local_policy is not None else '❌ 未加载'}")
print(f"   - 第二阶段（放置蓝杯）：{'✅ 云端policy' if cloud_policy is not None else '❌ 未加载，将使用本地policy'}")
print("=" * 50)

while PnPEnv.env.is_viewer_alive():
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # 检查当前任务是否完成
        success = PnPEnv.check_success()
        
        # 如果红块移除任务完成，切换到蓝杯放置任务
        if success and PnPEnv.current_task == 'remove_red_block':
            print('🎉 红块移除任务完成！')
            if PnPEnv.switch_to_next_task():
                # 重置云端policy状态，准备执行蓝杯放置任务
                if cloud_policy is not None:
                    cloud_policy.reset()
                    PnPEnv.policy_switch_count += 1
                    print(f"🔄 准备使用云端policy执行蓝杯放置任务 (切换次数: {PnPEnv.policy_switch_count})")
                    print("📝 确保使用正确的指令: 'Place the blue mug on the plate'")
                else:
                    print("⚠️ 警告：云端policy未加载，将使用本地policy执行蓝杯放置任务")
        
        # 检查所有任务是否都完成
        if PnPEnv.check_all_tasks_completed():
            print('🎊 所有任务完成！重置环境')
            # 完全重置所有policy状态
            local_policy.reset()
            if cloud_policy is not None:
                cloud_policy.reset()
            PnPEnv.reset()
            step = 0
            save_image = False
            print("🔄 开始新的双任务循环")
            continue
            
        # Get the current state of the environment
        state = PnPEnv.get_joint_state()[:6]
        # Get the current image from the environment
        image, wirst_image = PnPEnv.grab_image()
        image = Image.fromarray(image)
        image = image.resize((256, 256))
        image = IMG_TRANSFORM(image)
        wrist_image = Image.fromarray(wirst_image)
        wrist_image = wrist_image.resize((256, 256))
        wrist_image = IMG_TRANSFORM(wrist_image)
        
        # 根据当前任务直接选择对应的policy和指令
        if PnPEnv.current_task == 'place_blue_mug':
            # 第二阶段：使用云端policy执行蓝杯放置任务
            instruction = "Place the blue mug on the plate"
            selected_policy = cloud_policy if cloud_policy is not None else local_policy
            policy_name = "云端policy" if cloud_policy is not None else "本地policy(备用)"
        else:
            # 第一阶段：使用本地policy执行红块移除任务
            instruction = PnPEnv.instruction
            selected_policy = local_policy
            policy_name = "本地policy"
            
        data = {
            'observation.state': torch.tensor([state]).to(device),
            'observation.image': image.unsqueeze(0).to(device),
            'observation.wrist_image': wrist_image.unsqueeze(0).to(device),
            'task': [instruction],
        }
        
        # 直接使用对应任务的policy选择action
        action = selected_policy.select_action(data)
        action = action[0,:7].cpu().detach().numpy()
        
        # Take a step in the environment
        _ = PnPEnv.step(action)
        PnPEnv.render()
        step += 1
        
        # 显示当前任务状态
        if step % 50 == 0:  # 每50步显示一次状态
            print(f"步骤 {step}: 当前任务={PnPEnv.current_task}, 指令={instruction}")
            print(f"任务状态: 红块移除={PnPEnv.task_states['remove_red_block']}, 蓝杯放置={PnPEnv.task_states['place_blue_mug']}")
            print(f"实际使用的policy: {policy_name}")
            print(f"Policy切换次数: {PnPEnv.policy_switch_count}")
            
            # 检查指令是否包含可能冲突的关键词
            if 'red' in instruction.lower() and PnPEnv.current_task == 'place_blue_mug':
                print("⚠️  警告：蓝杯放置任务中检测到'red'关键词，可能导致policy混淆！")
            if 'mug' in instruction.lower() and PnPEnv.current_task == 'remove_red_block':
                print("⚠️  警告：红块移除任务中检测到'mug'关键词，可能导致policy混淆！")

DONE INITIALIZATION
🚀 开始执行双任务：1. 移除红块 2. 放置蓝杯
📋 Policy配置确认：
   - 第一阶段（移除红块）：✅ 本地policy
   - 第二阶段（放置蓝杯）：✅ 云端policy
步骤 50: 当前任务=remove_red_block, 指令=Remove the red block from the plate.
任务状态: 红块移除=False, 蓝杯放置=False
实际使用的policy: 本地policy
Policy切换次数: 0
步骤 100: 当前任务=remove_red_block, 指令=Remove the red block from the plate.
任务状态: 红块移除=False, 蓝杯放置=False
实际使用的policy: 本地policy
Policy切换次数: 0
步骤 150: 当前任务=remove_red_block, 指令=Remove the red block from the plate.
任务状态: 红块移除=False, 蓝杯放置=False
实际使用的policy: 本地policy
Policy切换次数: 0
步骤 200: 当前任务=remove_red_block, 指令=Remove the red block from the plate.
任务状态: 红块移除=False, 蓝杯放置=False
实际使用的policy: 本地policy
Policy切换次数: 0
步骤 250: 当前任务=remove_red_block, 指令=Remove the red block from the plate.
任务状态: 红块移除=False, 蓝杯放置=False
实际使用的policy: 本地policy
Policy切换次数: 0
步骤 300: 当前任务=remove_red_block, 指令=Remove the red block from the plate.
任务状态: 红块移除=False, 蓝杯放置=False
实际使用的policy: 本地policy
Policy切换次数: 0
步骤 350: 当前任务=remove_red_block, 指令=Remove the red block from the plate.


In [11]:
# policy.push_to_hub(
#     repo_id='Jeongeun/omy_pnp_smolvla',
#     commit_message='Add trained policy for PnP task',
# )